In [15]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
import glob

# Define the features and target columns
features = ['time', 'x0', 'y0', 'z0', 'x1', 'y1', 'z1', 'x2', 'y2', 'z2', 'x3', 'y3', 'z3', 
            'amp0', 'phase0', 'amp1', 'phase1', 'amp2', 'phase2', 'amp3', 'phase3']

# Define dropping_status columns
dropping_status_columns = ['drop_0', 'drop_1', 'drop_2', 'drop_3']

# Define the combined features list
all_features = features + dropping_status_columns

# File pattern to match different shots (feature files)
feature_file_pattern = 'C:/Users/weicheng/Desktop/formal_dataset/p4_x_r1_240813/4a-240702-gspat-crossing-0.50v-1*_pre_process_df2.csv'
feature_files = glob.glob(feature_file_pattern)

# Corresponding label file pattern
label_file_pattern = 'C:/Users/weicheng/Desktop/formal_dataset/p4_x_r1_240813/4a-240702-gspat-crossing-0.50v-1*_pre_process_df_labels.csv'
label_files = glob.glob(label_file_pattern)

# Initialize the scaler
scaler = MinMaxScaler(feature_range=(0, 1))

# Function to preprocess and create sequences for each shot
def preprocess_and_create_sequences(feature_file, label_file, time_steps=10):
    print(f"Processing: {feature_file}")
    
    # Read the feature data
    data = pd.read_csv(feature_file)
    
    # Read the corresponding label data
    labels = pd.read_csv(label_file)
    
    # Combine feature data with dropping status columns
    data_combined = pd.concat([data, labels[dropping_status_columns]], axis=1)

    # Handle missing values after combining the data
    # Interpolate NaNs if necessary
    data_combined.interpolate(method='linear', axis=0, inplace=True)
    data_combined.fillna(method='bfill', inplace=True)
    data_combined.fillna(method='ffill', inplace=True)

    # Scale the features including the dropping status columns
    scaled_data = scaler.fit_transform(data_combined[all_features])

    # Create sequences for time series data
    X, y = create_dataset(scaled_data, time_steps)
    return X, y

# Convert the data to a supervised learning problem
def create_dataset(data, time_steps=1):
    X, y = [], []
    for i in range(len(data) - time_steps):
        X.append(data[i:(i + time_steps), :])
        y.append(data[i + time_steps, :])  # Predicting the next time step
    return np.array(X), np.array(y)

# Define the number of time steps
time_steps = 10

# Prepare lists to hold model inputs and outputs
X_train_list, y_train_list = [], []
X_test_list, y_test_list = [], []

# Split files into training and testing sets (80% training, 20% testing)
train_files = feature_files[:int(len(feature_files) * 0.8)]
test_files = feature_files[int(len(feature_files) * 0.8):]
train_label_files = label_files[:int(len(label_files) * 0.8)]
test_label_files = label_files[int(len(label_files) * 0.8):]

# Process each training file separately
for feature_file, label_file in zip(train_files, train_label_files):
    X, y = preprocess_and_create_sequences(feature_file, label_file, time_steps)
    X_train_list.append(X)
    y_train_list.append(y)

# Process each testing file separately
for feature_file, label_file in zip(test_files, test_label_files):
    X, y = preprocess_and_create_sequences(feature_file, label_file, time_steps)
    X_test_list.append(X)
    y_test_list.append(y)

# Train the model on each shot separately.
for i, (X_train, y_train) in enumerate(zip(X_train_list, y_train_list)):
    print(f"Training on shot {i + 1}/{len(X_train_list)}")
    
    # Build the LSTM model
    model = Sequential()
    model.add(LSTM(50, return_sequences=True, input_shape=(time_steps, len(all_features))))
    model.add(LSTM(50))
    model.add(Dense(len(all_features)))  # Output layer predicting the same number of features

    model.compile(optimizer='adam', loss='mean_squared_error')

    # Train the model on the current shot's data
    model.fit(X_train, y_train, epochs=100, batch_size=32, validation_split=0.2)

# Evaluate the model on the test data from each shot
for i, (X_test, y_test) in enumerate(zip(X_test_list, y_test_list)):
    print(f"Evaluating on shot {i + 1}/{len(X_test_list)}")
    
    # Make predictions for the current test shot
    predictions = model.predict(X_test)

    # Inverse transform the predictions to the original scale
    scaled_predictions = np.concatenate((predictions, np.zeros((predictions.shape[0], len(all_features) - len(all_features)))), axis=1)
    predictions = scaler.inverse_transform(scaled_predictions)[:, :len(all_features)]

    # Save predictions to CSV for this specific shot
    output_df = pd.DataFrame(predictions, columns=all_features)
    
    output_df_1 = output_df[features[0:13]]
    
    # Assume output_df is already defined and you want to create a new DataFrame output_df_2
    output_df_2 = output_df[dropping_status_columns].applymap(lambda x: 1 if abs(x) > 0.5 else 0)
    
    output_df_1.to_csv(f'predictions_shot_df_{i + 1}.csv', index=False)
    output_df_2.to_csv(f'predictions_shot_df2_{i + 1}.csv', index=False)

    # print(f"Predictions for shot {i + 1}:")
    # print(predictions[:5])  # Print the first 5 predictions for review

# Predict on a new file (not in the training set)
# new_feature_file = 'C:/Users/weicheng/Desktop/formal_dataset/new_file.csv'
# new_label_file = 'C:/Users/weicheng/Desktop/formal_dataset/new_file_labels.csv'

# X_new, y_new = preprocess_and_create_sequences(new_feature_file, new_label_file, time_steps)

# # Make predictions for the new file
# new_predictions = model.predict(X_new)

# # Inverse transform the predictions to the original scale
# scaled_new_predictions = np.concatenate((new_predictions, np.zeros((new_predictions.shape[0], len(all_features) - len(all_features)))), axis=1)
# new_predictions = scaler.inverse_transform(scaled_new_predictions)[:, :len(all_features)]

# # Save new predictions to CSV
# new_output_df = pd.DataFrame(new_predictions, columns=all_features)

# new_output_df_1 = new_output_df[features]
# new_output_df_2 = new_output_df[dropping_status_columns].applymap(lambda x: 1 if abs(x) > 0.5 else 0)

# new_output_df_1.to_csv('new_file_predictions.csv', index=False)
# new_output_df_2.to_csv('new_file_predictions_dropping_status.csv', index=False)


Processing: C:/Users/weicheng/Desktop/formal_dataset/p4_x_r1_240813\4a-240702-gspat-crossing-0.50v-1_001_c_pre_process_df2.csv
Processing: C:/Users/weicheng/Desktop/formal_dataset/p4_x_r1_240813\4a-240702-gspat-crossing-0.50v-1_002_c_pre_process_df2.csv
Processing: C:/Users/weicheng/Desktop/formal_dataset/p4_x_r1_240813\4a-240702-gspat-crossing-0.50v-1_003_c_pre_process_df2.csv
Processing: C:/Users/weicheng/Desktop/formal_dataset/p4_x_r1_240813\4a-240702-gspat-crossing-0.50v-1_004_c_pre_process_df2.csv
Processing: C:/Users/weicheng/Desktop/formal_dataset/p4_x_r1_240813\4a-240702-gspat-crossing-0.50v-1_005_c_pre_process_df2.csv
Processing: C:/Users/weicheng/Desktop/formal_dataset/p4_x_r1_240813\4a-240702-gspat-crossing-0.50v-1_006_c_pre_process_df2.csv
Processing: C:/Users/weicheng/Desktop/formal_dataset/p4_x_r1_240813\4a-240702-gspat-crossing-0.50v-1_007_c_pre_process_df2.csv
Processing: C:/Users/weicheng/Desktop/formal_dataset/p4_x_r1_240813\4a-240702-gspat-crossing-0.50v-1_008_c_pre_

C:\Users\weicheng\AppData\Local\Temp\ipykernel_37592\1337917328.py:45: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  data_combined.fillna(method='bfill', inplace=True)
C:\Users\weicheng\AppData\Local\Temp\ipykernel_37592\1337917328.py:46: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  data_combined.fillna(method='ffill', inplace=True)
C:\Users\weicheng\AppData\Local\Temp\ipykernel_37592\1337917328.py:45: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  data_combined.fillna(method='bfill', inplace=True)
C:\Users\weicheng\AppData\Local\Temp\ipykernel_37592\1337917328.py:46: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  data_combin

Epoch 1/100
1/1 [==============================] - 12s 12s/step - loss: 0.2980 - val_loss: 0.4016
Epoch 2/100
1/1 [==============================] - 0s 41ms/step - loss: 0.2754 - val_loss: 0.3778
Epoch 3/100
1/1 [==============================] - 0s 38ms/step - loss: 0.2553 - val_loss: 0.3557
Epoch 4/100
1/1 [==============================] - 0s 41ms/step - loss: 0.2369 - val_loss: 0.3347
Epoch 5/100
1/1 [==============================] - 0s 36ms/step - loss: 0.2195 - val_loss: 0.3143
Epoch 6/100
1/1 [==============================] - 0s 35ms/step - loss: 0.2027 - val_loss: 0.2943
Epoch 7/100
1/1 [==============================] - 0s 36ms/step - loss: 0.1862 - val_loss: 0.2746
Epoch 8/100
1/1 [==============================] - 0s 35ms/step - loss: 0.1700 - val_loss: 0.2552
Epoch 9/100
1/1 [==============================] - 0s 36ms/step - loss: 0.1541 - val_loss: 0.2363
Epoch 10/100
1/1 [==============================] - 0s 35ms/step - loss: 0.1388 - val_loss: 0.2182
Epoch 11/100
1/1 [=

C:\Users\weicheng\AppData\Local\Temp\ipykernel_37592\1337917328.py:120: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  output_df_2 = output_df[dropping_status_columns].applymap(lambda x: 1 if abs(x) > 0.5 else 0)
C:\Users\weicheng\AppData\Local\Temp\ipykernel_37592\1337917328.py:120: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  output_df_2 = output_df[dropping_status_columns].applymap(lambda x: 1 if abs(x) > 0.5 else 0)
C:\Users\weicheng\AppData\Local\Temp\ipykernel_37592\1337917328.py:120: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  output_df_2 = output_df[dropping_status_columns].applymap(lambda x: 1 if abs(x) > 0.5 else 0)


2/2 [==============================] - 0s 14ms/step
Evaluating on shot 5/5
2/2 [==============================] - 0s 3ms/step


C:\Users\weicheng\AppData\Local\Temp\ipykernel_37592\1337917328.py:120: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  output_df_2 = output_df[dropping_status_columns].applymap(lambda x: 1 if abs(x) > 0.5 else 0)
C:\Users\weicheng\AppData\Local\Temp\ipykernel_37592\1337917328.py:120: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  output_df_2 = output_df[dropping_status_columns].applymap(lambda x: 1 if abs(x) > 0.5 else 0)
